# REQUEST by PREDICT


Copy-paste the request created in Predict

In [6]:
#Replace the two lines below with the result of your PREDICT request

req1 = " ( source_id == 'CESM2' ) & ( experiment_id == 'ssp585' ) & ( table_id == 'Amon' ) & ( variable_id == 'pr' ) "
req2 = " 'lons':[ -0.368031, 2.264177, 9.186397 ], 'lats':[ 39.445738, 48.827636, 45.458631 ] "

In [8]:
# Generate environment - it takes several minutes - If you run a second request, there is no need to run this piece of code again.
import pip
pip.main(["install","matplotlib", "pandas", "xarray", "zarr", "gcsfs", "cftime", "dask[array]", "toolz", "nc-time-axis", "openpyxl", " xlsxwriter"])

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import cftime
import xlsxwriter
import dask
import toolz
import os
import ast


xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

## Browse Catalog

The data catatalog is stored as a CSV file. Here we read it with Pandas.

In [4]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_ta = df.query(req1)


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,AerChemMIP,AS-RCEC,TaiESM1,histSST,r1i1p1f1,AERmon,od550aer,gn,gs://cmip6/AerChemMIP/AS-RCEC/TaiESM1/histSST/...,NaN,20200310
1,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20190718
2,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20191127
3,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20190809
4,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20191127


The columns of the dataframe correspond to the CMI6 controlled vocabulary. A beginners' guide to these terms is available in [this document](https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q). 

## Load Data

Now we will load a single store using gcsfs, zarr, and xarray.

In [102]:
df_out = {}
df_assets = {}
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('dataset_requested.xlsx', engine='xlsxwriter')


for i in range(df_ta.shape[0]):
    df_inter = []
    # this only needs to be created once
    gcs = gcsfs.GCSFileSystem(token='anon')

    # get the path to a specific zarr store (the first one from the dataframe above)
    zstore = df_ta.zstore.values[i]

    # create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(zstore)

    # open it using xarray and zarr
    ds = xr.open_zarr(mapper, consolidated=True)
    print(ds)
    
    #Get Metadata
    ds_attrs = pd.DataFrame.from_dict(ds.attrs, orient ='index')
    
    #Filter on assets
    # intialise data of lists. 
    df_assets = pd.DataFrame(ast.literal_eval(req2))
  
    for j in range(len(df_assets)):
        print(j)
        ds_sample = ds.sel(lon = df_assets["lons"].iloc[[j]], lat = df_assets["lats"].iloc[[j]], method = 'nearest')
        #Convert to df
        df_1 = ds_sample.to_dataframe().reset_index()
        print(df_1)
        if j == 0 :
            df_inter = df_1
            print(df_inter)
        else :
            df_inter = df_inter.append(df_1)
            print(df_inter)
    
    #Save in excel
    ds_attrs.to_excel(writer, sheet_name='Metadata_' + str(i))
    df_inter.to_excel(writer, sheet_name='Raw_Data_' + str(i))
    
writer.save()

<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2, time: 1980)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lat_bnds   (lat, nbnd) float32 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lon_bnds   (lon, nbnd) float32 dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    tas        (time, lat, lon) float32 dask.array<chunksize=(600, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  182500.0
    case_id:                18
    cesm_casename:          b.e21.BHIST.f09_g17.CMIP6-h